<a href="https://colab.research.google.com/github/srilamaiti/tf_learning/blob/main/Spotify_Baseline_Pegasus_Stride_05_Layer_1_Layer_2_no_ngram_repeate_part_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import locale
#locale.getpreferredencoding = lambda: "UTF-8"
!pip install -q sentencepiece
!pip install -q transformers

In [ ]:

!pip install -q evaluate
import evaluate

In [ ]:
!pip install -q rouge_score
!pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#let's make longer output readable without horizontal scrolling
from pprint import pprint

In [ ]:
import locale
import os
os.environ['LC_ALL'] = 'en_US.UTF-8'
os.environ['LANG'] = 'en_US.UTF-8'
locale.getpreferredencoding = lambda: "UTF-8"
#NLP related libraries
import transformers
print(f'transformers version: {transformers.__version__}')
from transformers import logging as hf_logging
#from transformers import BertTokenizer, TFBertModel
hf_logging.set_verbosity_error()
import nltk
from nltk.tokenize import sent_tokenize
from nltk import word_tokenize
from wordcloud import WordCloud
from wordcloud import STOPWORDS
from wordcloud import ImageColorGenerator
from sklearn.model_selection import train_test_split
nltk.download('punkt')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import PegasusTokenizer, PegasusForConditionalGeneration


# Other required libraries
#import os
import pandas as pd
import numpy as np
import re
import copy
import sys
import warnings
import io
import math
import torch
import json
warnings.filterwarnings("ignore")


# Data visualization libraries
import seaborn as sns
import matplotlib.pyplot as plt
import spacy
from joblib import Parallel, delayed
from scipy.spatial.distance import pdist, squareform



# Tensorflow libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.utils.layer_utils import count_params
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import LearningRateScheduler
from tensorflow.keras.losses import mae
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.losses import binary_crossentropy
from keras.models import Model
from tensorflow.keras import regularizers
from tensorflow.keras.regularizers import l1
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import SGD
from keras.models import load_model
os.environ['LC_ALL'] = 'en_US.UTF-8'
os.environ['LANG'] = 'en_US.UTF-8'




transformers version: 4.28.1


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Required to read the data from Kaggle
from google.colab import drive
from google.colab import files
drive.mount('/content/gdrive', force_remount=True)
os.environ['SPOTIFY_CONFIG_DIR'] = "/content/gdrive/MyDrive/Spotify"

print(os.environ.get('NotebookApp.iopub_data_rate_limit'))

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

Mounted at /content/gdrive
None


In [ ]:
import os
os.environ['LC_ALL'] = 'en_US.UTF-8'
os.environ['LANG'] = 'en_US.UTF-8'
#os.chdir('/content/drive/MyDrive/Spotify')
Original_path_file='/content/gdrive/MyDrive/Spotify/subsets/500/train.csv'


In [ ]:
import os

directory_path = "/content/gdrive/MyDrive/Spotify/subsets/Model_Output/Pegasus_Extractive_Abstractive_L2/"
file_count = len(os.listdir(directory_path))

print(f"Number of files in directory: {file_count}")


Number of files in directory: 430


In [ ]:
#df = pd.read_csv(io.BytesIO(Original_path_file))
 
# df.columns

with open(Original_path_file, 'rb') as f:
    data = io.BytesIO(f.read())
df_full = pd.read_csv(data)
print(len(df_full))



400


In [ ]:
df_full.head(100)


,Unnamed: 0,show_uri,show_name,show_description,publisher,language,rss_link,episode_uri,episode_name,episode_description,...,show_filename_prefix,episode_filename_prefix,word_count,cleaned_ep_desc,ID,Transcript,ep_desc_weirdchars,ep_desc_unidecoded,Transcript_weirdchars,Transcript_unidecoded
18,18,spotify:show:6R5ktHYRdfrm1ePuPcHeBe,Storytime with Jane,Listen to a new children’s storybook every day...,Jane,['en'],https://anchor.fm/s/126503d0/podcast/rss,spotify:episode:3jMe7zTRTwCm7q7VRqjDoX,The Gruffalo’s Child 📚,The little Gruffalo goes on an adventure throu...,...,show_6R5ktHYRdfrm1ePuPcHeBe,3jMe7zTRTwCm7q7VRqjDoX,59,The little Gruffalo goes on an adventure throu...,3jMe7zTRTwCm7q7VRqjDoX,Welcome to story time. Today. We're going to b...,❄️🐁🌳🌲🐿🦔’📚,Welcome to story time. Today. We're going to b...,NaN,Welcome to story time. Today. We're going to b...
22,22,spotify:show:5MK1Xk5Yd9ATgQGg7D3P1m,The Sweet Sounds of ASMR,Welcome to the Sweet Sounds Of ASMR for those ...,Parris Youngblood,['en'],https://anchor.fm/s/89553a0/podcast/rss,spotify:episode:0bLEUbqPdsFOEPck4G6FGM,ASMR of Things,In this ASMR podcast I will be putting makeup ...,...,show_5MK1Xk5Yd9ATgQGg7D3P1m,0bLEUbqPdsFOEPck4G6FGM,43,In this ASMR podcast I will be putting makeup ...,0bLEUbqPdsFOEPck4G6FGM,"Guys, do you want to hear something amazing? A...",,"Guys, do you want to hear something amazing? A...",NaN,"Guys, do you want to hear something amazing? A..."
50,50,spotify:show:5mlrDhLAEhaaGszJezXvjF,Daily Christian Podcast,Listen in on how reading the bible every day i...,Paul Dziepak,['en-US'],https://anchor.fm/s/2ec6a88/podcast/rss,spotify:episode:32tNQkYcpgFOmWOrqpvPnp,Reflect on your journey.,As we start reading through the book of Deuter...,...,show_5mlrDhLAEhaaGszJezXvjF,32tNQkYcpgFOmWOrqpvPnp,50,As we start reading through the book of Deuter...,32tNQkYcpgFOmWOrqpvPnp,It's pretty much the middle of the year right ...,NaN,It's pretty much the middle of the year right ...,NaN,It's pretty much the middle of the year right ...
52,52,spotify:show:4drmkFJHxGvu7X69hkjSUB,Affirmations by A.,Affirmations by A. delivers powerful affirmati...,Affirmations by A,['en'],https://anchor.fm/s/389bf18/podcast/rss,spotify:episode:1sYSra5nkgTMtnWuTq3yDA,34. Own Your Day,This episode was created to help you make a co...,...,show_4drmkFJHxGvu7X69hkjSUB,1sYSra5nkgTMtnWuTq3yDA,85,This episode was created to help you make a co...,1sYSra5nkgTMtnWuTq3yDA,"Hey guys, so if you haven't heard about anchor...",·,"Hey guys, so if you haven't heard about anchor...",NaN,"Hey guys, so if you haven't heard about anchor..."
53,53,spotify:show:1Ymux9Rk8sPpz7gtKdgeLR,Guru - The Alchemist,India market view Nifty Bank nifty Levels,MyBillion on the web,['en'],https://anchor.fm/s/c011628/podcast/rss,spotify:episode:7AUiSbcR4Rp8RgJvAS0I8i,India market view by Guru - The alchemist 27th...,"Nifty, bank nifty views, levels. Last pod cast...",...,show_1Ymux9Rk8sPpz7gtKdgeLR,7AUiSbcR4Rp8RgJvAS0I8i,42,"Nifty, bank nifty views, levels. Last pod cast...",7AUiSbcR4Rp8RgJvAS0I8i,Good morning friends. Welcome to report cause ...,NaN,Good morning friends. Welcome to report cause ...,NaN,Good morning friends. Welcome to report cause ...
60,60,spotify:show:0NpVLVfKd8mtKlBjmEj0vu,Stoic Meditations,Occasional reflections on the wisdom of Ancien...,Massimo Pigliucci,['en-US'],https://anchor.fm/s/1D80328/podcast/rss,spotify:episode:7Iq72L6XjQBFDZncbuO70i,Be careful with the company you keep,We should live with the quietest and easiest-t...,...,show_0NpVLVfKd8mtKlBjmEj0vu,7Iq72L6XjQBFDZncbuO70i,36,We should live with the quietest and easiest-t...,7Iq72L6XjQBFDZncbuO70i,Welcome to the stoic meditations podcast where...,NaN,Welcome to the stoic meditations podcast where...,NaN,Welcome to the stoic meditations podcast where...
72,72,spotify:show:3c3gzFgW09IjpruwOHoEaC,Steven's weekly podcast,Steven is a thought leader on the transformati...,Steven Van Belleghem,['nl-BE'],https://anchor.fm/s/3693b44/podcast/rss,spotify:episode

# Pegasus Model - PEGASUS model and tokenizer with chuck size of 150 reading first 10 and last 2 lines of the text 

In [ ]:

# Load the PEGASUS model and tokenizer with chuck size of 150 reading first 10 and last 2 lines of the text 
summary_model = 'google/pegasus-cnn_dailymail'
summary_tokenizer = PegasusTokenizer.from_pretrained(summary_model)
model = PegasusForConditionalGeneration.from_pretrained(summary_model)

def summarize_text(text):
    # Load the PEGASUS model and tokenizer
    summary_model = 'google/pegasus-xsum'
    summary_tokenizer = PegasusTokenizer.from_pretrained(summary_model)
    model = PegasusForConditionalGeneration.from_pretrained(summary_model)

    # Select the first 10 and last 2 lines of the text
    lines = text.split('\n')
    selected_lines = lines[:10] + lines[-2:]

    # Join the selected lines into a single string
    text = '\n'.join(selected_lines)

    # Encode the text into tokens
    tokens = summary_tokenizer.encode(text, add_special_tokens=False)

    # Split the tokens into chunks of up to 150 tokens
    chunk_size = 150
    chunks = [tokens[i:i+chunk_size] for i in range(0, len(tokens), chunk_size)]

    # Generate a summary for each chunk
    summaries = []
    for chunk in chunks:
        input_ids = torch.tensor(chunk).unsqueeze(0)
        summary_ids = model.generate(input_ids, num_beams=4, max_length=150, early_stopping=True)
        summary = summary_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        summaries.append(summary)

    # Combine the summaries into a single string
    summary = ' '.join(summaries)

    return summary


def evaluate_model(df):
    # Initialize the ROUGE scorer
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

    # Select the first 150 transcripts
    df = df.head(10)

    # Calculate the ROUGE scores for each row in the DataFrame
    rouge_scores = []
    for index, row in df.iterrows():
        if not row['Transcript']:  # Check if transcript column is empty
            continue
        # Generate the summary
        summary = summarize_text(row['Transcript'])
        
        # Calculate the ROUGE scores
        scores = scorer.score(row['episode_description'], summary)
        rouge_scores.append(scores)

    # Calculate the average ROUGE scores
    avg_scores = {}
    for metric in ['rouge1', 'rouge2', 'rougeL']:
        avg_scores[metric] = sum(score[metric].fmeasure for score in rouge_scores) / len(rouge_scores)

    return avg_scores





In [ ]:
scores = evaluate_model(selected_df)
print(scores)

{'rouge1': 0.10361759237705057, 'rouge2': 0.017757106981433783, 'rougeL': 0.06659571912393004}


In [ ]:
selected_df.head(100)
input_texts = selected_df["Transcript"].tolist()[:20]

# Pegasus Model - With error handling and trouble shooting

In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, pipeline


# Load the Pegasus model and tokenizer
model_name = "google/pegasus-xsum"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

# Define the summarization pipeline
summarizer = pipeline(
    "summarization",
    model=model,
    tokenizer=tokenizer,
    framework="pt",
    device=0,
)

# Read the input text from a file and summarize each transcript
input_df = pd.read_csv("/content/gdrive/MyDrive/Spotify/subsets/500/train.csv")
for i in range(len(input_df)):
    # Read the transcript
    transcript = input_df.loc[i, "Transcript"]
    
    # Generate the summary using Pegasus with a sliding window
    max_chunk = 512
    stride = 256
    output_text = ""
    print("Length of transcript", len(transcript))
    print("Transcript",transcript)
    for j in range(0, len(transcript), stride):
        chunk = transcript[j:j + max_chunk]
        if len(chunk) < max_chunk:
            break
        # Split the chunk into sentences and tokenize them
        sentences = chunk.split(".")
        encoded_sentences = tokenizer(sentences, padding=True, truncation=True, max_length=max_chunk, return_tensors="pt")
        input_ids = encoded_sentences["input_ids"].to("cuda")
        attention_mask = encoded_sentences["attention_mask"].to("cuda")
        try:
            # Generate the summary for the current chunk
            summary = summarizer(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=60,
                min_length=10,
                do_sample=False,
            )
            # Decode the summary and append it to the output text
            summary_text = tokenizer.decode(summary[0]["summary_text"])
            output_text += summary_text + " "
        except Exception as e:
            print(f"Error occurred while summarizing transcript {i}: {e}")
    
    # Write the summary to an output file
    output_dir = "/content/gdrive/MyDrive/Spotify/subsets/500/summaries" # specify the absolute path to the output directory
    os.makedirs(output_dir, exist_ok=True) # create the output directory if it does not exist
    output_filename = os.path.join(output_dir, f"summary_{i}.txt") # specify the absolute path to the output file
    with open(output_filename, "w") as f:
        f.write(output_text)


In [ ]:
#Read one file and write to the summaries directory train 
from transformers import PegasusTokenizer, PegasusForConditionalGeneration, SummarizationPipeline

# Load the Pegasus model and tokenizer
tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-xsum')
model = PegasusForConditionalGeneration.from_pretrained('google/pegasus-xsum')

# Define functions to read the transcript, summarize it, and write the summary to a file
def read_transcript(file_path):
    df = pd.read_csv(file_path)
    transcript = df['Transcript'][0]
    return transcript

def summarize_transcript(transcript, chunk_size=150, sliding_window=400):
    summarizer = SummarizationPipeline(
        model=model,
        tokenizer=tokenizer,
        device=0 if torch.cuda.is_available() else -1
    )
    chunks = [transcript[i:i+chunk_size] for i in range(0, len(transcript), sliding_window)]
    summaries = []
    for chunk in chunks:
        summary = summarizer(chunk, max_length=150, min_length=40, do_sample=False)[0]['summary_text']
        summaries.append(summary)
    summary = ' '.join(summaries)
    return summary

def write_summary_to_file(summary, file_path):
    with open(file_path, 'w') as f:
        f.write(summary)

# Call the functions to read the transcript, summarize it, and write the summary to a file
transcript = read_transcript('/content/gdrive/MyDrive/Spotify/subsets/500/train.csv')
summary = summarize_transcript(transcript, chunk_size=150, sliding_window=400)
write_summary_to_file(summary, '/content/gdrive/MyDrive/Spotify/subsets/500/summaries/train.csv')



In [ ]:

from transformers import PegasusTokenizer, PegasusForConditionalGeneration, SummarizationPipeline
import pandas as pd

# Load the Pegasus model and tokenizer
tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-xsum')
model = PegasusForConditionalGeneration.from_pretrained('google/pegasus-xsum')

# Define functions to read the transcript, summarize it, and write the summary to a file
def read_transcript(file_path):
    df = pd.read_csv(file_path)
    transcript = df['Transcript'][0]
    return transcript

def summarize_transcript(transcript, filename, file_path, chunk_size=150, sliding_window=400):
    summarizer = SummarizationPipeline(
        model=model,
        tokenizer=tokenizer,
        device=0 if torch.cuda.is_available() else -1
    )
    chunks = [transcript[i:i+chunk_size] for i in range(0, len(transcript), sliding_window)]
    summaries = []
    for chunk in chunks:
        summary = summarizer(chunk, max_length=150, min_length=40, do_sample=False)[0]['summary_text']
        summaries.append(summary)
    summary = ' '.join(summaries)
    write_summary_to_file(summary, filename, file_path)
    return summary

def write_summary_to_file(summary, filename, file_path):
    with open(file_path + filename + '.txt', 'w') as f:
        f.write(summary)

# Define start and end indices for the transcripts to summarize
start_idx = 1
end_idx = 2

# Loop over the specified range of transcripts and summarize each one
for i in range(start_idx - 1, end_idx):
    file_path = '/content/gdrive/MyDrive/Spotify/subsets/500/train.csv'
    filename = f'transcript_{i+1}'
    transcript = read_transcript(file_path)
    summary = summarize_transcript(transcript, filename, '/content/gdrive/MyDrive/Spotify/subsets/500/summaries/', chunk_size=150, sliding_window=400)
    print(f'Summarized transcript {i+1}/{end_idx-start_idx+1} and saved summary to /content/gdrive/MyDrive/Spotify/subsets/500/summaries/{filename}.csv')


Summarized transcript 1/2 and saved summary to /content/gdrive/MyDrive/Spotify/subsets/500/summaries/transcript_1.txt
Summarized transcript 2/2 and saved summary to /content/gdrive/MyDrive/Spotify/subsets/500/summaries/transcript_2.txt


In [ ]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration, SummarizationPipeline
import pandas as pd

# Load the Pegasus model and tokenizer
tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-xsum')
model = PegasusForConditionalGeneration.from_pretrained('google/pegasus-xsum')

# Define functions to read the transcript, summarize it, and write the summary to a file
def read_transcript(file_path):
    df = pd.read_csv(file_path)
    transcript = df['Transcript'][0]
    id = df['ID'][0]
    return transcript, id

def summarize_transcript(transcript, id, filename, file_path, chunk_size=150, sliding_window=400):
    summarizer = SummarizationPipeline(
        model=model,
        tokenizer=tokenizer,
        device=0 if torch.cuda.is_available() else -1
    )
    chunks = [transcript[i:i+chunk_size] for i in range(0, len(transcript), sliding_window)]
    summaries = []
    for chunk in chunks:
        summary = summarizer(chunk, max_length=150, min_length=40, do_sample=False)[0]['summary_text']
        summaries.append(summary)
    summary = ' '.join(summaries)
    write_summary_to_file(summary, id, filename, file_path)
    return summary

def write_summary_to_file(summary, id, filename, file_path):
    df = pd.DataFrame({
        'ID': [id],
        'Summary': [summary]
    })
    df.to_csv(file_path + filename + '.csv', index=False)

# Define start and end indices for the transcripts to summarize
start_idx = 126
end_idx = 136

# Loop over the specified range of transcripts and summarize each one
for i in range(start_idx - 1, end_idx):
    file_path = '/content/gdrive/MyDrive/Spotify/subsets/500/train.csv'
    filename = f'transcript_{i+1}'
    transcript, id = read_transcript(file_path)
    summary = summarize_transcript(transcript, id, filename, '/content/gdrive/MyDrive/Spotify/subsets/500/summaries/', chunk_size=150, sliding_window=400)
    print(f'Summarized transcript {i+1}/{end_idx-start_idx+1} and saved summary to /content/gdrive/MyDrive/Spotify/subsets/500/summaries/{filename}.csv')



Summarized transcript 126/11 and saved summary to /content/gdrive/MyDrive/Spotify/subsets/500/summaries/transcript_126.csv
Summarized transcript 127/11 and saved summary to /content/gdrive/MyDrive/Spotify/subsets/500/summaries/transcript_127.csv
Summarized transcript 128/11 and saved summary to /content/gdrive/MyDrive/Spotify/subsets/500/summaries/transcript_128.csv
Summarized transcript 129/11 and saved summary to /content/gdrive/MyDrive/Spotify/subsets/500/summaries/transcript_129.csv
Summarized transcript 130/11 and saved summary to /content/gdrive/MyDrive/Spotify/subsets/500/summaries/transcript_130.csv
Summarized transcript 131/11 and saved summary to /content/gdrive/MyDrive/Spotify/subsets/500/summaries/transcript_131.csv
Summarized transcript 132/11 and saved summary to /content/gdrive/MyDrive/Spotify/subsets/500/summaries/transcript_132.csv
Summarized transcript 133/11 and saved summary to /content/gdrive/MyDrive/Spotify/subsets/500/summaries/transcript_133.csv
Summarized trans

In [ ]:

from transformers import PegasusTokenizer, PegasusForConditionalGeneration, pipeline

# Load the Pegasus models and tokenizer for extractive and abstractive summarization
tokenizer_ext = PegasusTokenizer.from_pretrained('sshleifer/distill-pegasus-xsum-16-4', max_length=512,  device='cuda')
model_ext = PegasusForConditionalGeneration.from_pretrained('sshleifer/distill-pegasus-xsum-16-4', max_length=512)

tokenizer_abs = PegasusTokenizer.from_pretrained('google/pegasus-large', max_length=1024,   device='cuda')
model_abs = PegasusForConditionalGeneration.from_pretrained('google/pegasus-large',max_length=1024)

# Function to generate an extractive summary using sliding window with a frame size of 400 and chunk size of 150
def generate_extractive_summary(transcript, frame_size=400, chunk_size=150):
    summarizer = pipeline('summarization', model=model_ext, tokenizer=tokenizer_ext, device=0)
    num_frames = int(math.ceil(len(transcript) / float(frame_size)))
    summary = ""
    for i in range(num_frames):
        start = max(0, i * frame_size - chunk_size//2)
        end = min(len(transcript), (i+1)*frame_size + chunk_size//2)
        chunk = transcript[start:end]
        summary += summarizer(chunk, max_length=150, min_length=40, do_sample=False)[0]['summary_text']
    return summary

# Function to generate an abstractive summary using sliding window with a frame size of 400 and chunk size of 150
def generate_abstractive_summary(transcript, frame_size=400, chunk_size=150):
    summarizer = pipeline('summarization', model=model_abs, tokenizer=tokenizer_abs, device=0)
    num_frames = int(math.ceil(len(transcript) / float(frame_size)))
    summary = ""
    for i in range(num_frames):
        start = max(0, i * frame_size - chunk_size//2)
        end = min(len(transcript), (i+1)*frame_size + chunk_size//2)
        chunk = transcript[start:end]
        summary += summarizer(chunk, max_length=150, min_length=40, do_sample=False)[0]['summary_text']
    return summary




# Define a function to read the transcript from a CSV file and generate summaries
def generate_summaries(file_path, start_idx, end_idx, frame_size=400, chunk_size=150):
    df = pd.read_csv(file_path)
    processed = 0  # Initialize the number of transcripts processed
    for idx in range(start_idx, end_idx):
        transcript = df['Transcript'][idx]
        id = df['ID'][idx]
        num_frames = int(math.ceil(len(transcript) / float(frame_size)))
        extractive_summary = ""
        abstractive_summary = ""
        for i in range(num_frames):
            start = max(0, i * frame_size - chunk_size//2)
            end = min(len(transcript), (i+1)*frame_size + chunk_size//2)
            chunk = transcript[start:end]
            extractive_summary += generate_extractive_summary(chunk)
        abstractive_summary = generate_abstractive_summary(extractive_summary)
        summary = {"ID": id, "Extractive_Summary": extractive_summary, "Abstractive_Summary": abstractive_summary}
        # Write each summary and ID to individual JSON files
        with open(f"/content/gdrive/MyDrive/Spotify/subsets/500/Summary_hybrid_Pegasus/summary_{id}.json", 'w') as f:
            json.dump(summary, f)
        processed += 1  # Increment the number of transcripts processed
        print(f"Processed {processed} out of {end_idx-start_idx} transcripts")  # Print the progress update
    print(f"Generated and saved summaries for transcripts with IDs between {start_idx} and {end_idx}")


file_path='/content/gdrive/MyDrive/Spotify/subsets/500/train.csv'

# Define start and end indices for the transcripts to summarize
start_idx = 11
end_idx = 20

generate_summaries(file_path, start_idx, end_idx)


In [ ]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration, pipeline
import math

# Load the Pegasus models and tokenizer for extractive and abstractive summarization
tokenizer_ext = PegasusTokenizer.from_pretrained('sshleifer/distill-pegasus-xsum-16-4', max_length=512, device='cuda')
model_ext = PegasusForConditionalGeneration.from_pretrained('sshleifer/distill-pegasus-xsum-16-4', max_length=512)

tokenizer_abs = PegasusTokenizer.from_pretrained('google/pegasus-large', max_length=1024, device='cuda')
model_abs = PegasusForConditionalGeneration.from_pretrained('google/pegasus-large', max_length=1024)

# Define a function to generate an extractive summary for a given chunk of text
def generate_extractive_summary(transcript):
    summarizer = pipeline('summarization', model=model_ext, tokenizer=tokenizer_ext, device=0)
    max_chunk_size = 500
    chunks = [transcript[i:i+max_chunk_size] for i in range(0, len(transcript), max_chunk_size)]
    summary = ''
    for chunk in chunks:
        summary += summarizer(chunk, max_length=150, min_length=40, do_sample=False)[0]['summary_text']
    return summary

# Define a function to generate an abstractive  summary for a given chunk of text
def generate_abstractive_summary(transcript):
    summarizer = pipeline('summarization', model=model_abs, tokenizer=tokenizer_abs, device=0)
    max_chunk_size = 500
    chunks = [transcript[i:i+max_chunk_size] for i in range(0, len(transcript), max_chunk_size)]
    summary = ''
    for chunk in chunks:
        summary += summarizer(chunk, max_length=150, min_length=40, do_sample=False)[0]['summary_text']
    return summary


# Define a function to read the transcript from a CSV file and generate summaries
def generate_summaries(file_path, start_idx, end_idx, frame_size=400, chunk_size=150, stride_gap=-0.5):
    df = pd.read_csv(file_path)
    processed = 0  # Initialize the number of transcripts processed
    for idx in range(start_idx, end_idx):
        transcript = df['Transcript'][idx]
        id = df['ID'][idx]
        num_frames = int(math.ceil(len(transcript) / float(frame_size)))
        combined_summary = ""
        for i in range(num_frames):
            start = max(0, int(i * frame_size + stride_gap * chunk_size) - chunk_size//2)
            end = min(len(transcript), int((i+1)*frame_size + stride_gap * chunk_size) + chunk_size//2)
            chunk = transcript[start:end]
            summary = generate_extractive_summary(chunk)
            combined_summary += summary + " "
        final_summary = generate_abstractive_summary(combined_summary)
        summary = {"ID": id, "Extractive_Summary": combined_summary, "Abstractive_Summary": final_summary}
        # Write each summary and ID to individual JSON files
        with open("/content/gdrive/MyDrive/Spotify/subsets/500/Summary_hybrid_Pegasus/"f"summary_{id}.json", 'w') as f:
            json.dump(summary, f)
        processed += 1  # Increment the number of transcripts processed
        print(f"Processed {processed} out of {end_idx-start_idx} transcripts")  # Print the progress update
    print(f"Generated and saved summaries for transcripts with IDs between {start_idx} and {end_idx}")


file_path='/content/gdrive/MyDrive/Spotify/subsets/500/train.csv'

# Define start and end indices for the transcripts to summarize
start_idx = 11
end_idx = 20

generate_summaries(file_path, start_idx, end_idx)



Processed 1 out of 9 transcripts
Processed 2 out of 9 transcripts
Processed 3 out of 9 transcripts
Processed 4 out of 9 transcripts
Processed 5 out of 9 transcripts


# Pegasus Extractive Model Layer 1 with Rouge Scores  

In [ ]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration, pipeline
import math
from rouge import Rouge

# Load the Pegasus models and tokenizer for extractive summarization
tokenizer_ext = PegasusTokenizer.from_pretrained('google/pegasus-large', max_length=1024, device='cuda')
model_ext = PegasusForConditionalGeneration.from_pretrained('google/pegasus-large', max_length=1024)

# Define a function to generate an extractive summary for a given chunk of text
def generate_extractive_summary(transcript):
    summarizer = pipeline('summarization', model=model_ext, tokenizer=tokenizer_ext, device=0)
    max_chunk_size = 8192
    chunks = [transcript[i:i+max_chunk_size] for i in range(0, len(transcript), max_chunk_size)]
    summary = ''
    for chunk in chunks:
        summary += summarizer(chunk, max_length=150, min_length=40, do_sample=False, no_repeat_ngram_size=3)[0]['summary_text']
    return summary

# Define a function to calculate the Rouge score for a given summary and reference summary
def calculate_rouge(summary, reference_summary):
    rouge = Rouge()
    scores = rouge.get_scores(summary, reference_summary)[0]
    return scores

# Define a function to read the transcript from a CSV file and generate extractive summaries
def generate_extractive_summaries(transcript_file_path, metadata_file_path, start_idx, end_idx, frame_size=400, chunk_size=150, stride_gap=-0.5,no_repeat_ngram_size=3):
    df_transcript = pd.read_csv(transcript_file_path)
    df_metadata = pd.read_csv(metadata_file_path, delimiter="\t")
    processed = 0  # Initialize the number of transcripts processed
    for idx in range(start_idx, end_idx):
        transcript = df_transcript['Transcript'][idx]
        id = df_transcript['ID'][idx]
        num_frames = int(math.ceil(len(transcript) / float(frame_size)))
        combined_summary = ""
        for i in range(num_frames):
            start = max(0, int(i * frame_size + stride_gap * chunk_size) - chunk_size//2)
            end = min(len(transcript), int((i+1)*frame_size + stride_gap * chunk_size) + chunk_size//2)
            chunk = transcript[start:end]
            summary = generate_extractive_summary(chunk)
            combined_summary += summary + " "
        # Calculate Rouge score and add it to the summary dictionary
        reference_summary = df_metadata['episode_description'][idx]
        rouge_scores = calculate_rouge(combined_summary, reference_summary)
        summary = {"ID": id, "Extractive_Summary": combined_summary, "Rouge": rouge_scores}
        # Write each summary and ID to individual JSON files
        with open('/content/gdrive/MyDrive/Spotify/subsets/Pegasus_Extractive_L1/test/'f"summary_{id}.json", 'w') as f:
            json.dump(summary, f)
        processed += 1  # Increment the number of transcripts processed
        print(f"Processed {processed} out of {end_idx-start_idx} transcripts")  # Print the progress update
    print(f"Generated and saved extractive summaries for transcripts with IDs between {start_idx} and {end_idx}")

# Define start and end indices for the transcripts to summarize
start_idx = 0
end_idx=1

transcript_file_path='/content/gdrive/MyDrive/Spotify/subsets/500/test.csv'
metadata_file_path='/content/gdrive/MyDrive/Spotify/subsets/500/metadata.tsv'

generate_extractive_summaries(transcript_file_path, 
                              metadata_file_path, 
                              start_idx, 
                              end_idx, 
                              frame_size=500, 
                              chunk_size=250, 
                              stride_gap=-0.75, 
                              no_repeat_ngram_size=3)




Processed 1 out of 1 transcripts
Generated and saved extractive summaries for transcripts with IDs between 0 and 1


In [ ]:
# Load the metadata file
metadata_file_path = '/content/gdrive/MyDrive/Spotify/subsets/500/metadata.tsv'
df_metadata = pd.read_csv(metadata_file_path, delimiter='\t')

# Create an empty dataframe to store the IDs and descriptions
df_id = pd.DataFrame(columns=['id', 'episode_description'])

# Iterate over each row in the metadata file
for index, row in df_metadata.iterrows():
    episode_id = row['show_uri'].split(':')[2].lower()
    print(episode_id)
    # Get the corresponding episode_description
    episode_description = row['episode_description']
    print(episode_description)
    # Append the ID and episode_description to the df_id dataframe
    df_id = df_id.append({'id': episode_id, 'episode_description': episode_description}, ignore_index=True)

output_file_path = "/content/gdrive/MyDrive/Spotify/subsets/Pegasus_Extractive_L1/ids_metadata.tsv"
df_id.to_csv("/content/gdrive/MyDrive/Spotify/subsets/Pegasus_Extractive_L1/ids_metadata.tsv", sep='\t', index=False)




In [ ]:
metadata_file_path_id = '/content/gdrive/MyDrive/Spotify/subsets/Pegasus_Extractive_L1/ids_metadata.tsv'
df_metadata_id = pd.read_csv(metadata_file_path_id, delimiter="\t")
print(df_metadata_id.info())


id = "7nemplvhbs0suilrkkl3ia"  # example ID

if id in df_metadata_id["id"].values:
    print(f"Metadata found for ID {id}.")
else:
    print(f"No metadata found for ID {id}.")





<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105360 entries, 0 to 105359
Data columns (total 2 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   id                   105360 non-null  object
 1   episode_description  105155 non-null  object
dtypes: object(2)
memory usage: 1.6+ MB
None
No metadata found for ID 7nemplvhbs0suilrkkl3ia.


In [ ]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration, pipeline
import math
from rouge import Rouge
from tqdm import tqdm

# Load the Pegasus models and tokenizer for extractive summarization
tokenizer_ext = PegasusTokenizer.from_pretrained('google/pegasus-large', max_length=1024, device='cuda')
model_ext = PegasusForConditionalGeneration.from_pretrained('google/pegasus-large', max_length=1024)

# Define a function to generate an extractive summary for a given chunk of text
def generate_extractive_summary(transcript):
    summarizer = pipeline('summarization', model=model_ext, tokenizer=tokenizer_ext, device=0)
    max_chunk_size = 8192
    chunks = [transcript[i:i+max_chunk_size] for i in range(0, len(transcript), max_chunk_size)]
    summary = ''
    for chunk in chunks:
        summary += summarizer(chunk, max_length=150, min_length=40, do_sample=False, no_repeat_ngram_size=3)[0]['summary_text']
    return summary

# Define a function to calculate the Rouge score for a given summary and reference summary
def calculate_rouge(summary, reference_summary):
    rouge = Rouge()
    scores = rouge.get_scores(summary, reference_summary)[0]
    return scores

# Define a function to read the transcript from a CSV file and generate extractive summaries
def generate_extractive_summaries(transcript_file_path, metadata_file_path, start_idx, end_idx, frame_size=400, chunk_size=150, stride_gap=-0.5, no_repeat_ngram_size=3):
    df_transcript = pd.read_csv(transcript_file_path)
    df_metadata = pd.read_csv(metadata_file_path, delimiter="\t")
    processed = 0  # Initialize the number of transcripts processed
    for idx in range(start_idx, end_idx):
        transcript = df_transcript['Transcript'][idx]
        id = df_transcript['ID'][idx].lower()  # Convert ID to lowercase
        print(f"Processing transcript ID {id}")
        if id in df_metadata['id'].values:
            print(f"ID {id} found in metadata file.")
            reference_summary = df_metadata.loc[df_metadata['id'] == id, 'episode_description'].iloc[0]
        else:
            print(f"ID {id} not found in metadata file. Moving on to the next transcript.")
            continue
        num_frames = int(math.ceil(len(transcript) / float(frame_size)))
        combined_summary = ""
        for i in range(num_frames):
            start = max(0, int(i * frame_size + stride_gap * chunk_size) - chunk_size//2)
            end = min(len(transcript), int((i+1)*frame_size + stride_gap * chunk_size) + chunk_size//2)
            chunk = transcript[start:end]
            print("Chunk", chunk)
            summary = generate_extractive_summary(chunk)
            print("Summary",summary)
            combined_summary += summary + " "
        rouge_scores = calculate_rouge(combined_summary, reference_summary)
        summary = {"ID": id, "Extractive_Summary": combined_summary, "Rouge": rouge_scores}
        # Write each summary and ID to individual JSON files
        with open(f"/content/gdrive/MyDrive/Spotify/subsets/Pegasus_Extractive_L1/summary_{id}.json", 'w') as f:
            json.dump(summary, f)
        processed += 1  # Increment the number of transcripts processed
        print(f"Processed {processed} out of {end_idx-start_idx} transcripts")  # Print the progress update
    print(f"Generated and saved extractive summaries for transcripts with IDs between {start_idx} and {end_idx}")





# Define start and end indices for the transcripts to summarize
start_idx = 20
end_idx=30


transcript_file_path = '/content/gdrive/MyDrive/Spotify/subsets/Pegasus_Extractive_L1/extract.csv'
metadata_file_path = '/content/gdrive/MyDrive/Spotify/subsets/Pegasus_Extractive_L1/ids_metadata.tsv'


generate_extractive_summaries(transcript_file_path, 
                              metadata_file_path, 
                              start_idx, 
                              end_idx, 
                              frame_size=250, 
                              chunk_size=150, 
                              stride_gap=-0.50, 
                              no_repeat_ngram_size=3)





# Pegasus Layer 2 - Extractive + Abstractive with Rouge scores

In [ ]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration, pipeline
import math
from rouge import Rouge
import pandas as pd
import json

# Load the Pegasus models and tokenizer for extractive summarization
tokenizer_ext = PegasusTokenizer.from_pretrained('google/pegasus-large', max_length=1024, device='cuda')
model_ext = PegasusForConditionalGeneration.from_pretrained('google/pegasus-large', max_length=512)

# Load the Pegasus models and tokenizer for abstractive summarization
tokenizer_abs = PegasusTokenizer.from_pretrained('google/pegasus-large', max_length=1024)
model_abs = PegasusForConditionalGeneration.from_pretrained('google/pegasus-large')


# Define a function to generate an extractive summary for a given chunk of text
def generate_extractive_summary(transcript, model, tokenizer, extractive_chunk_size,no_repeat_ngram_size):
    summarizer = pipeline('summarization', model=model, tokenizer=tokenizer, device=0)
    max_chunk_size = 150
    chunks = [transcript[i:i+max_chunk_size] for i in range(0, len(transcript), max_chunk_size)]
    summary = ''
    for chunk in chunks:
        summary += summarizer(chunk, max_length=150, min_length=40, do_sample=False,no_repeat_ngram_size=3)[0]['summary_text']
    return summary


# Define a function to generate an abstractive summary for a given extractive summary
def generate_abstractive_summary(summary,model,tokenizer,no_repeat_ngram_size):
    max_chunk_size = 150
    chunks = [summary[i:i+max_chunk_size] for i in range(0, len(summary), max_chunk_size)]
    abstractive_summary = ''
    for chunk in chunks:
        input_ids = tokenizer_abs.encode(chunk, return_tensors='pt', truncation=True, max_length=1024)
        output = model_abs.generate(input_ids, max_length=128, num_beams=4, early_stopping=True,no_repeat_ngram_size=3)
        abstractive_summary += tokenizer_abs.decode(output[0], skip_special_tokens=True)
    return abstractive_summary


# Define a function to calculate the Rouge score for a given summary and reference summary
def calculate_rouge(summary, reference_summary):
    rouge = Rouge()
    scores = rouge.get_scores(summary, reference_summary)[0]
    return scores

def generate_summaries(transcript_file_path, metadata_file_path, start_idx, end_idx, frame_size=400, extractive_chunk_size=150, stride_gap=-0.5,no_repeat_ngram_size=3):

    df_transcript = pd.read_csv(transcript_file_path)
    df_metadata = pd.read_csv(metadata_file_path, delimiter="\t")
    processed = 0  # Initialize the number of transcripts processed
    for idx in range(start_idx, end_idx):
        transcript = df_transcript['Transcript'][idx]
        num_frames = int(math.ceil(len(transcript) / float(frame_size)))
        combined_summary = ""
        combined_summary_abs = ""
        for i in range(num_frames):
            start = max(0, int(i * frame_size + stride_gap * extractive_chunk_size) - extractive_chunk_size//2)
            end = min(len(transcript), int((i+1)*frame_size + stride_gap * extractive_chunk_size) + extractive_chunk_size//2)
            chunk = transcript[start:end]
            summary_ext = generate_extractive_summary(chunk, model_ext, tokenizer_ext, extractive_chunk_size,no_repeat_ngram_size)
            summary_abs = generate_abstractive_summary(summary_ext, model_abs, tokenizer_abs,no_repeat_ngram_size)
            combined_summary += summary_ext + " "
            combined_summary_abs += summary_abs + " "
        combined_summary = combined_summary.strip()  # Remove trailing whitespace
        combined_summary_abs = combined_summary_abs.strip()  # Remove trailing whitespace
        # Calculate Rouge scores and add them to the summary dictionary
        reference_summary = df_metadata['episode_description'][idx]
        rouge_scores_ext = calculate_rouge(combined_summary, reference_summary)
        rouge_scores_abs = calculate_rouge(combined_summary_abs, reference_summary)
        summary = {"ID": id, "Combined_Summary_Ext": combined_summary, "Combined_Summary_Abs": combined_summary_abs, "Rouge_Ext": rouge_scores_ext, "Rouge_Abs": rouge_scores_abs}
        # Write each summary and ID to individual JSON files
        with open('/content/gdrive/MyDrive/Spotify/subsets/500/summaries_train_pegasus_stride_05_abstractive/'f"summary_{id}.json", 'w') as f:
            json.dump(summary, f)
        processed += 1  # Increment the number of transcripts processed
        print(f"Processed {processed} out of {end_idx-start_idx} transcripts")  # Print the progress update
    print(f"Generated and saved summaries for transcripts with IDs between {start_idx} and {end_idx}")


    # Define start and end indices for the transcripts to summarize
start_idx = 0
end_idx = 2

# Define the file paths for the transcript and metadata files
transcript_file_path = '/content/gdrive/MyDrive/Spotify/subsets/500/train.csv'
metadata_file_path = '/content/gdrive/MyDrive/Spotify/subsets/500/metadata.tsv'

generate_summaries(transcript_file_path, metadata_file_path, start_idx, end_idx, frame_size=400, extractive_chunk_size=150, stride_gap=-0.5,no_repeat_ngram_size=3)


In [ ]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration, pipeline
import math
from rouge import Rouge
import pandas as pd
import json
# Load the Pegasus models and tokenizer for extractive summarization
tokenizer_ext = PegasusTokenizer.from_pretrained('google/pegasus-large',  max_length=1024,device='cuda')
model_ext = PegasusForConditionalGeneration.from_pretrained('google/pegasus-large', max_length=1024)
# Load the Pegasus models and tokenizer for abstractive summarization
tokenizer_abs = PegasusTokenizer.from_pretrained('google/pegasus-cnn_dailymail',  max_length=1024,device='cuda')
model_abs = PegasusForConditionalGeneration.from_pretrained('google/pegasus-cnn_dailymail', max_length=1024)
# Define a function to generate an extractive summary for a given chunk of text
def generate_extractive_summary(transcript):
    summarizer = pipeline('summarization', model=model_ext, tokenizer=tokenizer_ext, device=0)
    max_chunk_size = 8192
    chunks = [transcript[i:i+max_chunk_size] for i in range(0, len(transcript), max_chunk_size)]
    summary = ''
    for chunk in chunks:
        summary += summarizer(chunk, max_length=150, min_length=40, do_sample=False, no_repeat_ngram_size=3)[0]['summary_text']
    return summary
def generate_abstractive_summary(text, model, tokenizer, no_repeat_ngram_size, max_length=200):
    # Tokenize the input text
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=max_length, truncation=True)
    # Generate the abstractive summary
    summary_ids = model.generate(inputs,
                                 num_beams=4,
                                 length_penalty=0.8,
                                 min_length=75,
                                 max_length=200,
                                 no_repeat_ngram_size=2,
                                 early_stopping=False)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary
# Define a function to generate both extractive and abstractive summaries and compute the ROUGE scores
def generate_summaries(transcript_file_path, metadata_file_path, start_idx, stop_idx, model_abs, tokenizer_abs, model_ext, tokenizer_ext, frame_size=400, extractive_chunk_size=150, stride_gap=-0.5, no_repeat_ngram_size=3, abstractive_min_length=100, max_length=200):
    df_transcript = pd.read_csv(transcript_file_path)
    df_metadata = pd.read_csv(metadata_file_path, delimiter="\t")
    rouge = Rouge()
    rouge_scores = []
    processed = 0  # Initialize the number of transcripts processed
    for idx in range(start_idx, stop_idx):
        transcript = df_transcript['Transcript'][idx]
        print(f"The Original transcript ({len(transcript.split())} words): {transcript}")
        id = df_transcript['ID'][idx]
        num_frames = int(math.ceil(len(transcript) / float(frame_size)))
        print("################################")
        print(f"No of frames for Extractive Summary: {num_frames}")
        print(f"Transcript ID: {id}")
        summary_ext_chunks = []
        summary_abs_chunks = []
        for i in range(num_frames):
            start = max(0, int(i * frame_size + stride_gap * extractive_chunk_size) - extractive_chunk_size//2)
            end = min(len(transcript), int((i+1)*frame_size + stride_gap * extractive_chunk_size) + extractive_chunk_size//2)
            chunk = transcript[start:end]
            #print(f"Chunk {i+1} before extractive summary generation: {chunk}")
            summary_ext = generate_extractive_summary(chunk)
            summary_ext_chunks.append(summary_ext)
            #print(f"Extractive summary for chunk {i+1}: {summary_ext}")
        summary_ext = ' '.join(summary_ext_chunks)
        print(f"Final Extractive summary ({len(summary_ext.split())} words): {summary_ext}")
        combined_summary_abs_chunks = []
        num_frames_abs = int(math.ceil(len(summary_ext) / float(frame_size)))
        print("#################################")
        print(f"No of frames for Abstractive Summary: {num_frames_abs}")
        for i in range(num_frames_abs):
            start = max(0, int(i * frame_size + stride_gap * extractive_chunk_size) - extractive_chunk_size//2)
            end = min(len(summary_ext), int((i+1)*frame_size + stride_gap * extractive_chunk_size) + extractive_chunk_size//2)
            chunk = summary_ext[start:end]
            #print(f"Chunk {i+1} before abstractive summary generation: {chunk}")
            summary_abs = generate_abstractive_summary(chunk, model_abs, tokenizer_abs, no_repeat_ngram_size, max_length=max_length)
            combined_summary_abs_chunks.append(summary_abs)
            #print(f"Abstractive Summary for Chunk {i+1} : {summary_abs}")
        combined_summary_abs = ' '.join(combined_summary_abs_chunks)
        print(f"Final Abstractive summary  {combined_summary_abs}")
        # If the summary is too long, recursively generate a shorter summary
        if len(combined_summary_abs.split()) > abstractive_min_length:
            print(f"Abstractive summary before 100 words: {combined_summary_abs}")
            combined_summary_abs = combined_summary_abs.strip()  # Remove trailing space
            #print(f"\nCombined abstractive summary for transcript ID {id}: {combined_summary_abs}")
            # If the summary is too long, recursively generate a shorter summary
            while len(combined_summary_abs.split()) > abstractive_min_length:
              #print(f"Abstractive summary before {abstractive_min_length} words: {combined_summary_abs}")
              combined_summary_abs = generate_abstractive_summary(combined_summary_abs, model_abs, tokenizer_abs, no_repeat_ngram_size, max_length=max_length-10)
              #print(f"Abstractive summary after {abstractive_min_length} words: {combined_summary_abs}")
              combined_summary_abs = combined_summary_abs.strip()  # Remove trailing space
        print(f"\nFinal combined abstractive summary for transcript ID {id} ({len(combined_summary_abs.split())} words):: {combined_summary_abs}")
        # Compute the ROUGE scores for the extractive and abstractive summaries
        reference_summary = df_metadata['episode_description'][idx]
        summary_ext = summary_ext.replace('\n', ' ')
        summary_abs = summary_abs.replace('\n', ' ')
        combined_summary_abs = combined_summary_abs.replace('\n', ' ')
        rouge_scores.append({
            'ID': id,
            'Abstractive Summary': combined_summary_abs,
            'Abstractive ROUGE-1': rouge.get_scores(combined_summary_abs, reference_summary)[-1]['rouge-1']['f'],
            'Abstractive ROUGE-2': rouge.get_scores(combined_summary_abs, reference_summary)[-1]['rouge-2']['f'],
            'Abstractive ROUGE-L': rouge.get_scores(combined_summary_abs, reference_summary)[-1]['rouge-l']['f']
        })
        # Save the ROUGE scores to a JSON file for the current transcript ID
        with open('/content/gdrive/MyDrive/Spotify/subsets/Pegasus_Extractive_Abstractive_L2/'f"summary_{id}.json", 'w') as f:
            json.dump(rouge_scores, f)
          # Clear the rouge_scores list for the next transcript ID
        rouge_scores = []


# Define the file paths for the transcript and metadata files
transcript_file_path = '/content/gdrive/MyDrive/Spotify/subsets/Pegasus_Extractive_L1/valid.csv'
metadata_file_path = '/content/gdrive/MyDrive/Spotify/subsets/Pegasus_Extractive_L1/ids_metadata.tsv'
#target_id = '0wEYi9uEvNOetrxrIJ4wJq'


start_idx = 31
stop_idx = 41

# Call the generate_summaries method
generate_summaries(transcript_file_path, metadata_file_path, start_idx, stop_idx, model_abs, tokenizer_abs, model_ext, tokenizer_ext, frame_size=400, extractive_chunk_size=150, stride_gap=-0.5, no_repeat_ngram_size=3, abstractive_min_length=100, max_length=200)



In [ ]:
import json
import os

# Set the path to the directory containing the JSON files
dir_path = '/content/gdrive/MyDrive/Spotify/subsets/500/summaries_train_pegasus_stride_05'

# Initialize a dictionary to store the ROUGE scores for each metric
rouge_scores = {'rouge-1': [], 'rouge-2': [], 'rouge-l': []}

# Iterate over the files in the directory
for filename in os.listdir(dir_path):
    if filename.endswith('.json'):
        # Load the JSON file
        with open(os.path.join(dir_path, filename), 'r') as f:
            data = json.load(f)

        # Extract the ROUGE scores for each metric
        rouge_1_score = data['Rouge']['rouge-1']['f']
        rouge_2_score = data['Rouge']['rouge-2']['f']
        rouge_l_score = data['Rouge']['rouge-l']['f']

        # Append the ROUGE scores to the appropriate list in the dictionary
        rouge_scores['rouge-1'].append(rouge_1_score)
        rouge_scores['rouge-2'].append(rouge_2_score)
        rouge_scores['rouge-l'].append(rouge_l_score)

# Calculate the average ROUGE score for each metric
avg_rouge_1_score = sum(rouge_scores['rouge-1']) / len(rouge_scores['rouge-1'])
avg_rouge_2_score = sum(rouge_scores['rouge-2']) / len(rouge_scores['rouge-2'])
avg_rouge_l_score = sum(rouge_scores['rouge-l']) / len(rouge_scores['rouge-l'])

# Print the average ROUGE scores for each metric
print("Average ROUGE-1 score: {:.4f}".format(avg_rouge_1_score))
print("Average ROUGE-2 score: {:.4f}".format(avg_rouge_2_score))
print("Average ROUGE-L score: {:.4f}".format(avg_rouge_l_score))


Average ROUGE-1 score: 0.0761
Average ROUGE-2 score: 0.0083
Average ROUGE-L score: 0.0705


In [ ]:
import os

# Set the path to the directory containing the JSON files
dir_path = '/content/gdrive/MyDrive/Spotify/subsets/500/summaries_train_pegasus_stride_05'

# Initialize a counter variable
count = 0

# Iterate over the files in the directory
for filename in os.listdir(dir_path):
    if filename.endswith('.json'):
        # Increment the counter if the file is a JSON file
        count += 1

# Print the number of JSON files
print("Number of JSON files: ", count)


Number of JSON files:  374


In [ ]:
import csv

# Set the path to the CSV file
csv_path = '/content/gdrive/MyDrive/Spotify/subsets/500/train.csv'

# Initialize a counter variable
count = 0

# Open the CSV file with 'utf-8' encoding
with open(csv_path, 'r', encoding='utf-8') as csv_file:
    # Create a CSV reader object
    csv_reader = csv.reader(csv_file)

    # Iterate over the rows in the CSV file
    for row in csv_reader:
        # Increment the counter for each row
        count += 1

# Print the number of rows in the CSV file
print("Number of rows in the CSV file: ", count)



Number of rows in the CSV file:  401


In [ ]:
metadata_file_path = '/content/gdrive/MyDrive/Spotify/subsets/500/metadata.tsv'
df_metadata = pd.read_csv(metadata_file_path, delimiter="\t")
print(df_metadata.columns)

Index(['show_uri', 'show_name', 'show_description', 'publisher', 'language',
       'rss_link', 'episode_uri', 'episode_name', 'episode_description',
       'duration', 'show_filename_prefix', 'episode_filename_prefix'],
      dtype='object')


In [ ]:
import os
import json
import math
import pandas as pd
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from rouge import Rouge
import nltk

nltk.download('punkt')

# Load the Pegasus models and tokenizer for abstractive summarization
model_abs = PegasusForConditionalGeneration.from_pretrained('google/pegasus-large')
tokenizer_abs = PegasusTokenizer.from_pretrained('google/pegasus-large')

# Define a function to generate an abstractive summary for a given chunk of text
def generate_abstractive_summary(text):
    inputs = tokenizer_abs.encode(text, return_tensors='pt', truncation=True, max_length=1024)
    summary_ids = model_abs.generate(inputs, num_beams=4, length_penalty=2.0, max_length=256, early_stopping=True)
    summary = tokenizer_abs.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Define a function to calculate the Rouge score for a given summary and reference summary
def calculate_rouge(summary, reference_summary):
    rouge = Rouge()
    scores = rouge.get_scores(summary, reference_summary)[0]
    return scores

# Define a function to read the extractive summaries from JSON files, generate abstractive summaries and write the results to JSON files
def generate_abstractive_summaries(json_file_path, metadata_file_path, chunk_size, frame_size, stride_overlap):
    df_metadata = pd.read_csv(metadata_file_path, delimiter="\t")
    with open(json_file_path, 'r') as f:
        data = json.load(f)
        id = data['ID']
        extractive_summary = data['Extractive_Summary']
        sentences = nltk.sent_tokenize(extractive_summary)
        abstractive_summary = ""
        for i in range(0, len(sentences), int(len(sentences)*stride_overlap)):
            frame = sentences[i:i+frame_size]
            chunks = [frame[j:j+chunk_size] for j in range(0, len(frame), chunk_size)]
            for chunk in chunks:
                text = " ".join(chunk)
                summary = generate_abstractive_summary(text)
                abstractive_summary += summary.strip() + " "
        abstractive_summary = abstractive_summary.strip()
        reference_summary = df_metadata[df_metadata['ID'] == id]['episode_description'].values[0]
        rouge_scores = calculate_rouge(abstractive_summary, reference_summary)
        summary = {"ID": id, "Extractive_Summary": extractive_summary, "Abstractive_Summary": abstractive_summary, "Rouge": rouge_scores}
        with open('/content/gdrive/MyDrive/Spotify/subsets/500/summaries_train_pegasus_stride_05_abstractive/'f"summary_{id}.json", 'w') as f:
            json.dump(summary, f)
        print(f"Generated abstractive summary for transcript ID {id}")



# Define the file paths and IDs of the transcripts to summarize
json_file_path = '/content/gdrive/MyDrive/Spotify/subsets/500/summaries_train_pegasus_stride_05/'
metadata_file_path = '/content/gdrive/MyDrive/Spotify/subsets/500/metadata.tsv'

# Specify the number of JSON files to process at a time
batch_size = 5

# Get a list of all JSON files in the specified directory
json_files = [os.path.join(json_file_path, f) for f in os.listdir(json_file_path) if f.endswith('.json')]


# Loop through all JSON files and generate abstractive summaries for each batch
for i in range(0, len(json_files), batch_size):
    batch_files = json_files[i:i+batch_size]
    for json_file in batch_files:
        generate_abstractive_summaries(json_file, metadata_file_path, 200, 400, -0.5)


df_metadata = pd.read_csv(metadata_file_path, delimiter="\t")
print(df_metadata.columns)



In [ ]:
import os
import pandas as pd

def extract_rows_with_ids(ids, input_file_path, output_file_path, directory_path):
    """
    Extracts the rows with the given transcript IDs from the input file and writes them to a new CSV file.

    Parameters:
    ids (list): A list of transcript IDs to extract.
    input_file_path (str): The path to the input file.
    output_file_path (str): The path to the output file.
    directory_path (str): The path to the directory containing the files with the transcript IDs as suffixes.

    Returns:
    None
    """
    # check if each ID in the list has a corresponding file in the directory
    for id in ids:
        file_path = os.path.join(directory_path, f"{id}.csv")
        if not os.path.isfile(file_path):
            raise ValueError(f"No file found for ID {id} in directory {directory_path}")
    
    # read the csv file into a pandas dataframe
    df = pd.read_csv(input_file_path)
    
    # lowercase the IDs in the list
    ids = [id.lower() for id in ids]
    
    # lowercase the IDs in the 'ID' column of the dataframe
    df['ID'] = df['ID'].str.lower()
    
    # filter the dataframe to include only the rows with the given IDs
    filtered_df = df[df['ID'].isin(ids)]
    
    # write the filtered dataframe to a new CSV file
    filtered_df.to_csv(output_file_path, index=False)
    
    # print a message indicating the output file path
    print(f"Extracted rows with IDs {ids} and saved to {output_file_path}")

# Example usage:
ids_to_extract = [
    
    ]
input_file_path = '/content/gdrive/MyDrive/Spotify/subsets/500/train.csv'
output_file_path = '/content/gdrive/MyDrive/Spotify/subsets/Pegasus_Extractive_L1/extract.csv'
directory_path = '/content/gdrive/MyDrive/Spotify/subsets/Model_Output/Pegasus_Extractive_Abstractive_L2/'

# extract the rows with the given IDs and write the resulting dataframe to a CSV file
extract_rows_with_ids(ids_to_extract, input_file_path, output_file_path, directory_path)


In [ ]:
import os

# Set the directory path
dir_path = '/content/gdrive/MyDrive/Spotify/subsets/Model_Output/Pegasus_Extractive_Abstractive_L2/'

# Loop through all files in the directory
for filename in os.listdir(dir_path):
    
    # Check if the file is in the correct format
    if filename.endswith('.json') and filename.startswith('summary_'):
        
        # Get the ID from the filename
        id = filename[len('summary_'):-len('.json')]
        
        # Print the ID
        print(id)


7CcAQwKpwz7W7Tybbj97Km
57bfPuB7VYdpv55KsWhEbT
6YeWUBptv0OHarljT8Wwrf
5ZApaov7Xx7ntTVlfpdAxY
28CAQkt2GblBPK4vo4aE32
5gq7aguWdeGnJfW55oburg
7oUo2HJANZlErbhC5y3xN1
7dcD8SwiaikzogvWbdCuPM
58rD5mU99LDGYeNxaPTOOF
2ZNfMu94hWwMIaE5xf0DMr
5GDrhuy2K2Om2l3KyDjutK
3jte3EHLegwz7QrmQQfd7L
0uzsB2RHbmWxICsNi5DyXd
3w26WUIXyCQ1suqaKbWjuo
6lAC1hLD7uODrjBp8FRmjw
28X7VZX6kIGj9PNMLto7EK
3P0sUNoADKL9HPolfIgTK5
7vq9Fid8DS6NMUQwdpAXZg
6e6Tqpy9gQehm3BsR7h5bg
1eYf1f2rACWiIIBVj4v77w
3J1dmb2HsnL7nctr1wpjX5
3jMe7zTRTwCm7q7VRqjDoX
2eAmE2HF4GEi5yXpgKkWng
6KnWFL81JMaovhI4CEt8gH
7nEMPlVhBS0SUilrkKl3ia
0bLEUbqPdsFOEPck4G6FGM
7yYi8cywoIxckyBnCRCKDN
5NpNNtR0gx0tO5CCEEbOCg
5eJPzdD8uU8yPpBHqQjPYS
5yRyiAxHPGLChBqTfT3FXp
4ISXhcr9fwxRcfCEDR2wiH
6kqFQCO7wXieZUopdVsf9c
4o5pSHhPy6n7NorTb9kSVu
11KOktqHfuReK6qINXyE6r
6kKfjQbg8ueAIQyIUkcoBQ
5xYnrq3zbJhNgNQQYZ1cZk
5LFtjocUcricEq9tJ5O5WP
0IGtnJ2dQ73cZgvOkJS7HM
6kZoeBWA7m5XhLjmsJGAZx
1fgi6DTfcBv13S9ZVizlmz
5O9Wetc78TGfPHcVCq8RnY
26VtfRMBZw8feaMlDW9KDP
1TNxTloyo4qMgcpQVKBE2A
2riWgHX0yjg

In [ ]:


import os

dir_path = '/content/gdrive/MyDrive/Spotify/subsets/Model_Output/Pegasus_Extractive_Abstractive_L2/'
 # Replace with your directory path

file_count = len([f for f in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, f))])

print(f"Number of files in {dir_path}: {file_count}")






Number of files in /content/gdrive/MyDrive/Spotify/subsets/Model_Output/Pegasus_Extractive_Abstractive_L2/: 500
